In [25]:
import json
import threading
import json
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from urllib.request import urlopen
from PIL import Image
from skimage.io import imread
from skimage.transform import resize

from keras.utils.data_utils import Sequence
from keras.callbacks import ModelCheckpoint   
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, GlobalAveragePooling2D, GlobalMaxPooling2D, MaxPooling2D
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions

DATA_DIR = "../data/"
NUM_CLASSES = 228
IMAGE_SIZE = 197

In [12]:
with open(DATA_DIR + "test.json") as test:
    test_json = json.load(test)
    
test_urls = [obj['url'] for obj in test_json['images']]

print(test_urls[:3])

['https://contestimg.wish.com/api/webimage/568e16a72dfd0133cb3f7a79-large', 'https://contestimg.wish.com/api/webimage/5452f9925f313f502bf119ff-large', 'https://contestimg.wish.com/api/webimage/540584051d2d435c5a300a82-large']


In [10]:
conv_base = ResNet50(
    weights='imagenet',
    input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3),
    include_top = False,
    classes = NUM_CLASSES
)

model = Sequential()
model.add(conv_base)
model.add(GlobalMaxPooling2D())
model.add(Dropout(0.3))
model.add(Dense(30, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(NUM_CLASSES, activation='softmax'))
conv_base.trainable = False
print(model.summary())

model.load_weights(DATA_DIR + "model.best.hdf5")

model.compile(
    loss='categorical_crossentropy', 
    optimizer='rmsprop', 
    metrics=['accuracy']
)

94658560/94653016 [==============================] - 50s 1us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1, 1, 2048)        23587712  
_________________________________________________________________
global_max_pooling2d_3 (Glob (None, 2048)              0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 30)                61470     
_________________________________________________________________
dropout_6 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 228)               7068      
Total params: 23,656,250
Trainable params: 68,538
Non-trainable params: 23,5

In [13]:
class TestBatchSequence(Sequence):
    def __init__(self, x_set, batch_size):
        self.x = x_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        images = [self.url_to_image(file_name) for file_name in batch_x]
        return np.array(images), np.zeros(IMAGE_SIZE, IMAGE_SIZE, 1)
    
    def url_to_image(self, url):
        try:
            resp = urllib.urlopen(url)
            image = np.asarray(bytearray(resp.read()), dtype='uint8')
            image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        except:
            output = [1]*(IMAGE_SIZE*IMAGE_SIZE*3)
            output = np.array(output).reshape(IMAGE_SIZE,IMAGE_SIZE,3).astype('uint8')
            image = Image.fromarray(output).convert('RGB')
        image = np.array(image)
        image = resize(image, (IMAGE_SIZE, IMAGE_SIZE))
        return image

In [64]:
def url_to_image(url):
    try:
        resp = urlopen(url)
        image = np.asarray(bytearray(resp.read()), dtype='uint8')
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    except Exception as e: 
        print(e)
        output = [1]*(IMAGE_SIZE*IMAGE_SIZE*3)
        output = np.array(output).reshape(IMAGE_SIZE,IMAGE_SIZE,3).astype('uint8')
        image = Image.fromarray(output).convert('RGB')        
    image = image[...,::-1]
    image = np.array(image)
    image = resize(image, (IMAGE_SIZE, IMAGE_SIZE))    
    return image

In [ ]:
with open("submission.csv","w") as f:
    for i, image_url in enumerate(test_urls):
        image = url_to_image(image_url)
        prob = model.predict_proba(image.reshape(1, IMAGE_SIZE, IMAGE_SIZE, 3))
        sorted_args = np.argsort(prob)[0][::-1][:10]
        output_labels = " ".join(str(x) for x in sorted_args)
        # print("{}: {}".format(i + 1, output_labels))
        f.write("{}, {}\n".format(i + 1, output_labels))

/Users/matt/dev/ML/kaggle-fashion/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Users/matt/dev/ML/kaggle-fashion/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
